In [2]:
checkins = sc.textFile('../YelpDataset/yelp_academic_dataset_checkin.json')
businesses = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')

In [3]:
import operator
import math
import json

def mapper_checkins(records):
    for line in records:
        line = json.loads(line)
        checkin_info = line['checkin_info']
        for key, val in checkin_info.iteritems():
            yield (line['business_id'], val)

        
total = checkins.mapPartitions(mapper_checkins).reduceByKey(operator.add)
total.take(5)


[(u's5yzZITWU_RcJzWOgjFecw', 125),
 (u'F7i6y29C51ZEgMv5Qyx0pw', 3),
 (u'z_LY6PK0Vv-iLQxrIGQIWw', 19),
 (u'nZww1gdBAi9HtRKiQHL2qA', 74),
 (u'j13Aby6-9ZyklpG_W7qsew', 7)]

In [4]:
selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']
def mapper_business(records):
    for line in records:
        line = json.loads(line)
        category = line['categories']
        for entry in category:
            if entry in selectedCategories:
                yield (line['business_id'], entry)

another_total = businesses.mapPartitions(mapper_business)
#another_total[:5]
#another_total.count()

In [5]:
join = another_total.join(total).map(lambda x: (x[1][0], (x[0], x[1][1]))).groupByKey().mapValues(list)

In [6]:
import operator

def mapper_transformation(records):
    for category in records:
        maxCheckins = max(category[1], key=operator.itemgetter(1))
        minCheckins = min(category[1], key=operator.itemgetter(1))
        # yield (maxCheckins, minCheckins)
#         k = []
        for item in category[1]:
            normalizedTotal = map(lambda tup: (tup[0], float(tup[1])), item)
            yield normalizedTotal
#             normalizedTotal = map(lambda x: (item[0], (float(x[1]) - minCheckins)/(maxCheckins - minCheckins) * 50), item)
#             k.append(normalizedTotal)
#         yield (category, k)
        
another_total = join.mapPartitions(mapper_transformation)
another_total.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 11, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lukas/Desktop/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/lukas/Desktop/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/lukas/Desktop/spark-1.6.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/lukas/Desktop/spark-1.6.1-bin-hadoop2.6/python/pyspark/rdd.py", line 1293, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-6-e